In [ ]:
# Khai báo sử dụng các thư viện. Thực hiện chạy ngay trên CPU, do đó, chỉ thực hiện mô phỏng, nếu ổn định sẽ thực hiện chạy trên kaggle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import regex as re
import os
import sys
import hashlib
from datetime import datetime
import hashlib
from tabulate import tabulate
import copy

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
import importlib
import Drain_test
importlib.reload(Drain_test)

input_dir  = './PreTrain0'   # Đường dẫn vào thư mục chứa file đầu vào
output_dir = './res'                      # Thư mục kết quả
log_file   = 'filelog1.log'                 # Tên file đầu vào
log_format = '<Date> <Time>,<Pid> <Level> <Component>: <Content>'  # HDFS log format
# Biểu thức chính quy

regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Ngưỡng tương đồng
depth      = 4  # Độ sâu của cây phân tích

rex_unix = "%Y-%m-%d %H:%M:%S"
row_dt = "<Date> <Time>"
parserObj = Drain_test.LogParserEdit(
    log_format, indir=input_dir, outdir=output_dir, depth=depth, st=st, rex=regex,
    rex_unix=rex_unix, row_dt=row_dt,
    )
# parserObj.parse(log_file)

# selected_columns = ['LineId', 'UnixTime', 'Date', 'Time', 'EventId', 'EventIdVN', 'EventTemplate', 'ParameterList']

parserObj.parse("filelog2.log") 

# for i in range(1,2):
#     str_file = "filelog"+str(i)+".log"
#     parserObj.parse(str_file)
#     print(len(parserObj.list_logClust))
#     # parserObj.df_log = pd.DataFrame()
#     parserObj.df_log = None

In [ ]:
import pickle

# Lưu cây vào file
with open("tree.pkl", "wb") as f:
    pickle.dump(parserObj.rootNode, f)
# Lưu cây vào file
with open("logCluL.pkl", "wb") as f:
    pickle.dump(parserObj.list_logClust, f)

# Đọc lại cây từ file
with open("tree.pkl", "rb") as f:
    loaded_tree = pickle.load(f)

parserObj2 = Drain_test.LogParserEdit(
    log_format, indir=input_dir, outdir=output_dir, depth=depth, st=st, rex=regex,
    rootNode=loaded_tree,
    rex_unix=rex_unix, row_dt=row_dt,
    )

with open("logCluL.pkl", "rb") as f:
    parserObj2.list_logClust = pickle.load(f)

parserObj2.parse("filelog2.log") 
print(len(parserObj2.list_logClust))